### Import necessary libraries 

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import warnings 
warnings.simplefilter(action='ignore')

### Load Dataset

In [2]:
df=pd.read_csv('movies.csv')
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


### Data cleaning and visualization

In [3]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [4]:
max_val=df['budget'].max()
max_val

380000000

In [5]:
df['scaled_budget']=df['budget'].apply(lambda x: x/max_val)

In [6]:
use_cols=['genres','scaled_budget','keywords','title','cast','director','overview']

In [7]:
df1=df[use_cols]

In [8]:
df1.isnull().sum()

genres            28
scaled_budget      0
keywords         412
title              0
cast              43
director          30
overview           3
dtype: int64

In [9]:
df1.head()

,genres,scaled_budget,keywords,title,cast,director,overview
0,Action Adventure Fantasy Science Fiction,0.623684,culture clash future space war space colony so...,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,"In the 22nd century, a paraplegic Marine is di..."
1,Adventure Fantasy Action,0.789474,ocean drug abuse exotic island east india trad...,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski,"Captain Barbossa, long believed to be dead, ha..."
2,Action Adventure Crime,0.644737,spy based on novel secret agent sequel mi6,Spectre,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes,A cryptic message from Bond’s past sends him o...
3,Action Crime Drama Thriller,0.657895,dc comics crime fighter terrorist secret ident...,The Dark Knight Rises,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan,Following the death of District Attorney Harve...
4,Action Adventure Science Fiction,0.684211,based on novel mars medallion space travel pri...,John Carter,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton,"John Carter is a war-weary, former military ca..."


In [10]:
df1.fillna(' ',inplace=True)

In [11]:
combined_feature=df1['scaled_budget'].astype('str')+' '+df1['genres']+' '+df1['keywords']+' '+df1['title']+' '+df1['director']+' '+df1['overview']+' '+df1['cast']

In [12]:
combined_feature

0       0.6236842105263158 Action Adventure Fantasy Sc...
1       0.7894736842105263 Adventure Fantasy Action oc...
2       0.6447368421052632 Action Adventure Crime spy ...
3       0.6578947368421053 Action Crime Drama Thriller...
4       0.6842105263157895 Action Adventure Science Fi...
                              ...                        
4798    0.0005789473684210527 Action Crime Thriller un...
4799    2.368421052631579e-05 Comedy Romance   Newlywe...
4800    0.0 Comedy Drama Romance TV Movie date love at...
4801    0.0     Shanghai Calling Daniel Hsia When ambi...
4802    0.0 Documentary obsession camcorder crush drea...
Length: 4803, dtype: object

In [13]:
df1[df1['scaled_budget']==0.0]

,genres,scaled_budget,keywords,title,cast,director,overview
265,Comedy Fantasy Family,0.0,cat brother sister relationship boredom chaos ...,The Cat in the Hat,Mike Myers Dakota Fanning Spencer Breslin Kell...,Bo Welch,Conrad and Sally Walden are home alone with th...
321,Comedy,0.0,politics politician election campaign \u00a0no...,The Campaign,Will Ferrell Zach Galifianakis Dylan McDermott...,Jay Roach,Two rival politicians compete to win an electi...
359,Adventure Animation Comedy Family,0.0,chipmunk cgi talking animal aftercreditsstinge...,Alvin and the Chipmunks: The Road Chip,Jason Lee Justin Long Bella Thorne Matthew Gra...,Walt Becker,"Through a series of misunderstandings, Alvin, ..."
406,Drama Animation Family Comedy,0.0,holiday santa claus duringcreditsstinger woman...,Arthur Christmas,James McAvoy Hugh Laurie Bill Nighy Jim Broadb...,Barry Cook,"Each Christmas, Santa and his vast army of hig..."
409,Drama Music,0.0,show business film making tap dancing movie in...,All That Jazz,Roy Scheider Jessica Lange Leland Palmer Ann R...,Bob Fosse,Bob Fosse's semi-autobiographical film celebra...
...,...,...,...,...,...,...,...
4795,Drama,0.0,gang audition police fake homeless actress,Bang,Darling Narita Peter Greene Michael Newland Er...,Ash Baron-Cohen,A young woman in L.A. is having a bad day: she...
4797,Foreign Thriller,0.0,,Cavite,,Neill Dela Llana,"Adam, a security guard, travels from Californi..."
4800,Comedy Drama Romance TV Movie,0.0,date love at first sight narration investigati...,"Signed, Sealed, Delivered",Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith,"""Signed, Sealed, Delivered"" introduces a dedic..."
4801,,0.0,,Shanghai Calling,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia,When ambitious New York attorney Sam is sent t...


In [14]:
df1[df1['scaled_budget']==0.0].shape 

#Remove scaled_budget because of huge variation 

(1037, 7)

In [15]:
combined_feature=df1['genres']+' '+df1['keywords']+' '+df1['title']+' '+df1['director']+' '+df1['overview']+' '+df1['cast']

In [16]:
combined_feature

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance   Newlyweds Edward Burns A newl...
4800    Comedy Drama Romance TV Movie date love at fir...
4801        Shanghai Calling Daniel Hsia When ambitiou...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

### Feature Engineering

In [17]:
vectorizer=TfidfVectorizer()
feature_extraction=vectorizer.fit_transform(combined_feature)

In [18]:
print(feature_extraction)

  (0, 23378)	0.15712983615485113
  (0, 17961)	0.12880237917518134
  (0, 15734)	0.17965748183948915
  (0, 26145)	0.1215164162454309
  (0, 29933)	0.15793829476600177
  (0, 25067)	0.16661467904553673
  (0, 23824)	0.17614923402628724
  (0, 30862)	0.15793829476600177
  (0, 30462)	0.19126567940650283
  (0, 23866)	0.11581852375116888
  (0, 5303)	0.17784484373422185
  (0, 1000)	0.12826618032438802
  (0, 1236)	0.05652328304013453
  (0, 21745)	0.17965748183948915
  (0, 1274)	0.03376550889063298
  (0, 19854)	0.1716342411380125
  (0, 10462)	0.1456927977062822
  (0, 3004)	0.10813666397755077
  (0, 27808)	0.1541239522466706
  (0, 2709)	0.10340830620353815
  (0, 4170)	0.06637540405604263
  (0, 18205)	0.1159842834919336
  (0, 28900)	0.16339356886372833
  (0, 19745)	0.05259466095062308
  (0, 20246)	0.19436017328657268
  :	:
  (4802, 19444)	0.06090505416661404
  (4802, 12059)	0.07831113162150243
  (4802, 29996)	0.07417215993777906
  (4802, 9428)	0.07273191935436099
  (4802, 19684)	0.05495347694155716
  

In [19]:
movie_name=input("Enter the movie_name:")
similar_movies=difflib.get_close_matches(movie_name,list(df.title),n=3)


Enter the movie_name: avatar


In [20]:
movie_index=df1.index[df1.title==similar_movies[0]]
movie_index.tolist()

[0]

In [21]:
similarity=cosine_similarity(feature_extraction)
len(similarity)


4803

In [22]:
similarity_score=list(enumerate(similarity[movie_index[0]]))
similarity_score[0:9]

[(0, 1.0),
 (1, 0.03330113121811146),
 (2, 0.03100455879489204),
 (3, 0.031289371034825576),
 (4, 0.07617507130196044),
 (5, 0.05164737593355014),
 (6, 0.01607170090990939),
 (7, 0.062114666919639426),
 (8, 0.020662640954680074)]

In [23]:
sorted_score=sorted(similarity_score,key=lambda x:x[1],reverse=True)
sorted_score[0:9]

[(0, 1.0),
 (1914, 0.16066063846566153),
 (461, 0.15435260416602975),
 (56, 0.14976212443415304),
 (1352, 0.14864119364513492),
 (1531, 0.14843292113325568),
 (94, 0.14733554848118408),
 (3158, 0.1472835647052912),
 (2403, 0.14571660223784694)]

In [24]:
a=int(input("Enter the movie_count you want for recommendation :" ))
print(f"The recommendation for {a} is listed")
for i in sorted_score[0:a]:
      display(df1.iloc[i[0],3])


Enter the movie_count you want for recommendation : 10


The recommendation for 10 is listed


'Avatar'

'Lifeforce'

'Lost in Space'

...

'Moonraker'

'Guardians of the Galaxy'

'Alien'

'Aliens'

'Zathura: A Space Adventure'

### Output

In [ ]:
movie_name=input("Enter the movie_name:")
similar_movies=difflib.get_close_matches(movie_name,list(df.title),n=3)
movie_index=df1.index[df1.title==similar_movies[0]]
movie_index.tolist()
similarity=cosine_similarity(feature_extraction)
similarity_score=list(enumerate(similarity[movie_index[0]]))
sorted_score=sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(f"The recommendation for {a} similar movies is listed")
for i in sorted_score[0:7]:
      display(df1.iloc[i[0],3])

Enter the movie_name: avatar


The recommendation for 10 similar movies is listed


'Avatar'

'Lifeforce'

'Lost in Space'

'Star Trek Beyond'

'Gattaca'

'Moonraker'

In [ ]:
import pickle 
pickle.dump(df1,open('artifacts/movie_lst.pkl','wb'))
pickle.dump(df1,open('artifacts/similarity_score.pkl','wb'))
